# 0. Imports and Data Load

In [9]:
import pandas as pd
import classifiers
import importlib
from igraph import Graph
import numpy as np
from ast import literal_eval
np.seterr(divide='ignore', invalid='ignore')

{'divide': 'ignore', 'over': 'warn', 'under': 'ignore', 'invalid': 'ignore'}

In [69]:
hypergeometric = pd.read_csv("../../data/processed/metrics/disease_hypergeometric_apid_huri.csv", sep=',', header=0, index_col=0)
closeness = pd.read_csv("../../data/processed/metrics/disease_closeness_apid_huri.csv", sep=',', header=0, index_col=0)
betweenness = pd.read_csv("../../data/processed/metrics/disease_betweenness_apid_huri.csv", sep=',', header=0, index_col=0)
rwr = pd.read_csv("../../data/processed/metrics/disease_rwr_apid_huri.csv", sep=',', header=0, index_col=0)
fraction_betweenness = pd.read_csv("../../data/processed/metrics/disease_fraction_betweenness_apid_huri.csv", sep=',', header=0, index_col=0)

hypergeometric_80 = pd.read_csv("../../data/processed/metrics/disease_ppi80_hyper_apid_huri.csv", sep=',', header=0, index_col=0)
closeness_80 = pd.read_csv("../../data/processed/metrics/disease_ppi80_closeness_apid_huri.csv", sep=',', header=0, index_col=0)
betweenness_80 = pd.read_csv("../../data/processed/metrics/disease_ppi80_betweenness_apid_huri.csv", sep=',', header=0, index_col=0)
fraction_betweenness_80 = pd.read_csv("../../data/processed/metrics/disease_ppi80_fraction_betweenness_apid_huri.csv", sep=',', header=0, index_col=0)
rwr_80 = pd.read_csv("../../data/processed/metrics/disease_ppi80_rwr_apid_huri.csv", sep=',', header=0, index_col=0)

hypergeometric_protein80 = pd.read_csv("../../data/processed/metrics/disease_protein80_hyper_apid_huri.csv", sep=',', header=0, index_col=0)
closeness_protein80 = pd.read_csv("../../data/processed/metrics/disease_protein80_closeness_apid_huri.csv", sep=',', header=0, index_col=0)
betweenness_protein80 = pd.read_csv("../../data/processed/metrics/disease_protein80_betweenness_apid_huri.csv", sep=',', header=0, index_col=0)
fraction_betweenness_protein80 = pd.read_csv("../../data/processed/metrics/disease_protein80_fraction_betweenness_apid_huri.csv", sep=',', header=0, index_col=0)
rwr_protein80 = pd.read_csv("../../data/processed/metrics/disease_protein80_rwr_apid_huri.csv", sep=',', header=0, index_col=0)

In [33]:
hypergeometric_conservative = pd.read_csv("../../data/processed/metrics/disease_hypergeometric_conservative_apid_huri.csv", sep=',', header=0, index_col=0)
closeness_conservative = pd.read_csv("../../data/processed/metrics/disease_closeness_conservative_apid_huri.csv", sep=',', header=0, index_col=0)
betweenness_conservative = pd.read_csv("../../data/processed/metrics/disease_betweenness_conservative_apid_huri.csv", sep=',', header=0, index_col=0)
#rwr_conservative = pd.read_csv("../../data/processed/metrics/disease_rwr_conservative_apid_huri.csv", sep=',', header=0, index_col=0)
#fraction_betweenness_conservative = pd.read_csv("../../data/processed/metrics/disease_fraction_betweenness_conservative_apid_huri.csv", sep=',', header=0, index_col=0)

#hypergeometric_conservative_80 = pd.read_csv("../../data/processed/metrics/disease_ppi80_hyper_conservative_apid_huri.csv", sep=',', header=0, index_col=0)
#closeness_conservative_80 = pd.read_csv("../../data/processed/metrics/disease_ppi80_closeness_conservative_apid_huri.csv", sep=',', header=0, index_col=0)
#betweenness_conservative_80 = pd.read_csv("../../data/processed/metrics/disease_ppi80_betweenness_conservative_apid_huri.csv", sep=',', header=0, index_col=0)
#fraction_betweenness_conservative_80 = pd.read_csv("../../data/processed/metrics/disease_ppi80_fraction_betweenness_conservative_apid_huri.csv", sep=',', header=0, index_col=0)
#rwr_conservative_80 = pd.read_csv("../../data/processed/metrics/disease_ppi80_rwr_conservative_apid_huri.csv", sep=',', header=0, index_col=0)

#hypergeometric_conservative_protein80 = pd.read_csv("../../data/processed/metrics/disease_protein80_hyper_conservative_apid_huri.csv", sep=',', header=0, index_col=0)
#closeness_conservative_protein80 = pd.read_csv("../../data/processed/metrics/disease_protein80_closeness_conservative_apid_huri.csv", sep=',', header=0, index_col=0)
#betweenness_conservative_protein80 = pd.read_csv("../../data/processed/metrics/disease_protein80_betweenness_conservative_apid_huri.csv", sep=',', header=0, index_col=0)
#fraction_betweenness_conservative_protein80 = pd.read_csv("../../data/processed/metrics/disease_protein80_fraction_betweenness_conservative_apid_huri.csv", sep=',', header=0, index_col=0)
#rwr_conservative_protein80 = pd.read_csv("../../data/processed/metrics/disease_protein80_rwr_conservative_apid_huri.csv", sep=',', header=0, index_col=0)

In [13]:
graph = Graph.Read_GML("../../data/processed/graph_apid_huri")
disgenet_proteins_indexes_df = pd.read_csv("../../data/processed/disgenet_prot_index_main_comp.csv", sep=',', header=0)

disgenet_proteins_indexes_df['protein_index'] = disgenet_proteins_indexes_df['protein_index'].apply(literal_eval)
disgenet_proteins_indexes_df['main_component'] = disgenet_proteins_indexes_df['main_component'].apply(literal_eval)
disgenet_proteins_indexes_df['conservative_module'] = disgenet_proteins_indexes_df['conservative_module'].apply(literal_eval)
disgenet_proteins_indexes_df['added_nodes'] = disgenet_proteins_indexes_df['added_nodes'].apply(literal_eval)

disgenet_prot_index_main_comp = disgenet_proteins_indexes_df[disgenet_proteins_indexes_df['process'].isin(hypergeometric.columns)]

In [29]:
disgenet_labels = np.zeros(hypergeometric.shape)
column = 0
for indexes in disgenet_prot_index_main_comp['main_component'].values:
    for pos in indexes:
        disgenet_labels[pos][column] = 1
    column += 1
np.savetxt("../../data/processed/disgenet_filtered_labels_apid_huri.csv", disgenet_labels, delimiter=",")

In [34]:
disgenet_labels_conservative = np.zeros(hypergeometric_conservative.shape)
column = 0
for indexes in disgenet_proteins_indexes_df['conservative_module'].values:
    for pos in indexes:
        disgenet_labels_conservative[pos][column] = 1
    column += 1
np.savetxt("../../data/processed/disgenet_conservative_labels_apid_huri.csv", disgenet_labels_conservative, delimiter=",")

In [40]:
disgenet_labels_df = pd.read_csv("../../data/processed/disgenet_filtered_labels_apid_huri.csv", sep=',', names=disgenet_prot_index_main_comp['process'].values)
disgenet_labels_df['protein_id'] = graph.vs['name']
disgenet_labels_df.set_index('protein_id', inplace=True)

In [42]:
disgenet_labels_conservative_df = pd.read_csv('../../data/processed/disgenet_conservative_labels_apid_huri.csv', names=disgenet_proteins_indexes_df['process'].values)
disgenet_labels_conservative_df['protein_id'] = graph.vs['name']
disgenet_labels_conservative_df.set_index('protein_id', inplace=True)

In [86]:
ppi80_hyper_list = []
ppi80_closeness_list = []
ppi80_betweenness_list = []
ppi80_fraction_betweenness_list = []
ppi80_rwr_list = []
label_list = []

for i, df in hypergeometric_80.groupby(level=0):
    df.set_index('level_1', inplace=True)
    ppi80_hyper_list.append(df)
    label_list.append(disgenet_labels_df)

for i, df in closeness_80.groupby(level=0):
    df.set_index('level_1', inplace=True)
    ppi80_closeness_list.append(df)
    
for i, df in betweenness_80.groupby(level=0):
    df.set_index('level_1', inplace=True)
    ppi80_betweenness_list.append(df)
    
for i, df in fraction_betweenness_80.groupby(level=0):
    df.set_index('index', inplace=True)
    ppi80_fraction_betweenness_list.append(df)
    
for i, df in rwr_80.groupby(level=0):
    df.set_index('level_1', inplace=True)
    ppi80_rwr_list.append(df)

In [90]:
protein80_hyper_list = []
protein80_closeness_list = []
protein80_betweenness_list = []
protein80_fraction_betweenness_list = []
protein80_rwr_list = []
label_list_protein80 = []

for i, df in hypergeometric_protein80.groupby(level=0):
    df.set_index('level_1', inplace=True)
    protein80_hyper_list.append(df)
    label_list_protein80.append(disgenet_labels_df)

for i, df in closeness_protein80.groupby(level=0):
    df.set_index('level_1', inplace=True)
    protein80_closeness_list.append(df)
    
for i, df in betweenness_protein80.groupby(level=0):
    df.set_index('level_1', inplace=True)
    protein80_betweenness_list.append(df)
    
for i, df in fraction_betweenness_protein80.groupby(level=0):
    df.set_index('index', inplace=True)
    protein80_fraction_betweenness_list.append(df)
    
for i, df in rwr_protein80.groupby(level=0):
    df.set_index('level_1', inplace=True)
    protein80_rwr_list.append(df)

In [ ]:
ppi80_hyper_list_conservative = []
ppi80_closeness_list_conservative = []
ppi80_betweenness_list_conservative = []
ppi80_fraction_betweenness_list_conservative = []
ppi80_rwr_list_conservative = []
label_list_conservative = []

for i, df in hypergeometric_conservative_80.groupby(level=0):
    df.set_index('level_1', inplace=True)
    ppi80_hyper_list_conservative.append(df)
    label_list_conservative.append(disgenet_labels_conservative_df)

for i, df in closeness_conservative_80.groupby(level=0):
    df.set_index('level_1', inplace=True)
    ppi80_closeness_list_conservative.append(df)
    
for i, df in betweenness_conservative_80.groupby(level=0):
    df.set_index('level_1', inplace=True)
    ppi80_betweenness_list_conservative.append(df)
    
for i, df in fraction_betweenness_conservative_80.groupby(level=0):
    df.set_index('level_1', inplace=True)
    ppi80_fraction_betweenness_list_conservative.append(df)
    
for i, df in rwr_conservative_80.groupby(level=0):
    df.set_index('level_1', inplace=True)
    ppi80_rwr_list_conservative.append(df)

In [ ]:
protein80_hyper_list_conservative = []
protein80_closeness_list_conservative = []
protein80_betweenness_list_conservative = []
protein80_fraction_betweenness_list_conservative = []
protein80_rwr_list_conservative = []
label_list_protein80_conservative = []

for i, df in hypergeometric_conservative_protein80.groupby(level=0):
    df.set_index('level_1', inplace=True)
    protein80_hyper_list_conservative.append(df)
    label_list_protein80_conservative.append(disgenet_labels_conservative_df)

for i, df in closeness_conservative_protein80.groupby(level=0):
    df.set_index('level_1', inplace=True)
    protein80_closeness_list_conservative.append(df)
    
for i, df in betweenness_conservative_protein80.groupby(level=0):
    df.set_index('level_1', inplace=True)
    protein80_betweenness_list_conservative.append(df)
    
for i, df in fraction_betweenness_conservative_protein80.groupby(level=0):
    df.set_index('level_1', inplace=True)
    protein80_fraction_betweenness_list_conservative.append(df)
    
for i, df in rwr_conservative_protein80.groupby(level=0):
    df.set_index('level_1', inplace=True)
    protein80_rwr_list_conservative.append(df)

In [92]:
hypergeometric_test_indices = pd.read_csv("../../data/processed/fs/disease/disease_hyper_test_apid_huri.csv", sep=',', header=0, dtype='Int64', names=hypergeometric.columns)-1
closeness_test_indices = pd.read_csv("../../data/processed/fs/disease/disease_closeness_test_apid_huri.csv", sep=',', header=0, dtype='Int64', names=closeness.columns)-1
betweenness_test_indices = pd.read_csv("../../data/processed/fs/disease/disease_betweenness_test_apid_huri.csv", sep=',', header=0, dtype='Int64', names=betweenness.columns)-1
fraction_betweenness_test_indices = pd.read_csv("../../data/processed/fs/disease/disease_fraction_betweenness_test_apid_huri.csv", sep=',', header=0, dtype='Int64', names=fraction_betweenness.columns)-1
rwr_test_indices = pd.read_csv("../../data/processed/fs/disease/disease_rwr_test_apid_huri.csv", sep=',', header=0, dtype='Int64', names=rwr.columns)-1

hypergeometric_80_test_index = np.array(np.array_split(pd.read_csv("../../data/processed/fs/disease/disgenet_hyper_ppi80_test_apid_huri.csv", sep=',', header=0).transpose(), 10, axis = 1)).astype(int) -1
closeness_80_test_index = np.array(np.array_split(pd.read_csv("../../data/processed/fs/disease/disgenet_closeness_ppi80_test_apid_huri.csv", sep=',', header=0).transpose(), 10, axis = 1)).astype(int) -1
betweenness_80_test_index = np.array(np.array_split(pd.read_csv("../../data/processed/fs/disease/disgenet_betweenness_ppi80_test_apid_huri.csv", sep=',', header=0).transpose(), 10, axis = 1)).astype(int) -1
fraction_betweenness_80_test_index = np.array(np.array_split(pd.read_csv("../../data/processed/fs/disease/disgenet_fraction_betweenness_ppi80_test_apid_huri.csv", sep=',', header=0).transpose(), 10, axis = 1)).astype(int) -1
rwr_80_test_index = np.array(np.array_split(pd.read_csv("../../data/processed/fs/disease/disgenet_rwr_ppi80_test_apid_huri.csv", sep=',', header=0).transpose(), 10, axis = 1)).astype(int) -1

hypergeometric_protein80_test_index = np.array(np.array_split(pd.read_csv("../../data/processed/fs/disease/disgenet_hyper_protein80_test_apid_huri.csv", sep=',', header=0).transpose(), 10, axis = 1)).astype(int) -1
closeness_protein80_test_index = np.array(np.array_split(pd.read_csv("../../data/processed/fs/disease/disgenet_closeness_protein80_test_apid_huri.csv", sep=',', header=0).transpose(), 10, axis = 1)).astype(int) -1
betweenness_protein80_test_index = np.array(np.array_split(pd.read_csv("../../data/processed/fs/disease/disgenet_betweenness_protein80_test_apid_huri.csv", sep=',', header=0).transpose(), 10, axis = 1)).astype(int) -1
fraction_betweenness_protein80_test_index = np.array(np.array_split(pd.read_csv("../../data/processed/fs/disease/disgenet_fraction_betweenness_protein80_test_apid_huri.csv", sep=',', header=0).transpose(), 10, axis = 1)).astype(int) -1
rwr_protein80_test_index = np.array(np.array_split(pd.read_csv("../../data/processed/fs/disease/disgenet_rwr_protein80_test_apid_huri.csv", sep=',', header=0).transpose(), 10, axis = 1)).astype(int) -1

In [ ]:
#hypergeometric_test_indices_conservative = pd.read_csv("../../data/processed/fs/disease/disease_hyper_test_apid_huri.csv", sep=',', header=0, dtype='Int64', names=hypergeometric.columns)-1
#closeness_test_indices_conservative = pd.read_csv("../../data/processed/fs/disease/disease_closeness_test_apid_huri.csv", sep=',', header=0, dtype='Int64', names=closeness.columns)-1
#betweenness_test_indices = pd.read_csv("../../data/processed/fs/disease/disease_betweenness_test_apid_huri.csv", sep=',', header=0, dtype='Int64', names=betweenness.columns)-1
#fraction_betweenness_test_indices_conservative = pd.read_csv("../../data/processed/fs/disease/disease_fraction_betweenness_test_apid_huri.csv", sep=',', header=0, dtype='Int64', names=fraction_betweenness.columns)-1
#rwr_test_indices_conservative = pd.read_csv("../../data/processed/fs/disease/disease_rwr_test_apid_huri.csv", sep=',', header=0, dtype='Int64', names=rwr.columns)-1

#hypergeometric_80_test_index_conservative = np.array(np.array_split(pd.read_csv("../../data/processed/fs/disease/disease_hyper_ppi80_test_apid_huri.csv", sep=',', header=0).transpose(), 429, axis = 1)).astype(int) -1
#closeness_80_test_index_conservative = np.array(np.array_split(pd.read_csv("../../data/processed/fs/disease/disease_closeness_ppi80_test_apid_huri.csv", sep=',', header=0).transpose(), 429, axis = 1)).astype(int) -1
#betweenness_80_test_index_conservative = np.array(np.array_split(pd.read_csv("../../data/processed/fs/disease/disease_betweenness_ppi80_test_apid_huri.csv", sep=',', header=0).transpose(), 429, axis = 1)).astype(int) -1
#fraction_betweenness_80_test_index_conservative = np.array(np.array_split(pd.read_csv("../../data/processed/fs/disease/disease_fraction_betweenness_ppi80_test_apid_huri.csv", sep=',', header=0).transpose(), 429, axis = 1)).astype(int) -1
#rwr_80_test_index_conservative = np.array(np.array_split(pd.read_csv("../../data/processed/fs/disease/disease_rwr_ppi80_test_apid_huri.csv", sep=',', header=0).transpose(), 429, axis = 1)).astype(int) -1

#hypergeometric_protein80_test_index_conservative = np.array(np.array_split(pd.read_csv("../../data/processed/fs/disease/disease_hyper_protein80_test_apid_huri.csv", sep=',', header=0).transpose(), 429, axis = 1)).astype(int) -1
#closeness_protein80_test_index_conservative = np.array(np.array_split(pd.read_csv("../../data/processed/fs/disease/disease_closeness_protein80_test_apid_huri.csv", sep=',', header=0).transpose(), 429, axis = 1)).astype(int) -1
#betweenness_protein80_test_index_conservative = np.array(np.array_split(pd.read_csv("../../data/processed/fs/disease/disease_betweenness_protein80_test_apid_huri.csv", sep=',', header=0).transpose(), 429, axis = 1)).astype(int) -1
#fraction_betweenness_protein80_test_index_conservative = np.array(np.array_split(pd.read_csv("../../data/processed/fs/disease/disease_fraction_betweenness_protein80_test_apid_huri.csv", sep=',', header=0).transpose(), 429, axis = 1)).astype(int) -1
#rwr_protein80_test_index_conservative = np.array(np.array_split(pd.read_csv("../../data/processed/fs/disease/disease_rwr_protein80_test_apid_huri.csv", sep=',', header=0).transpose(), 429, axis = 1)).astype(int) -1

# 1. Threshold Classifier - Complete Network

## 1.1. Classification with Steiner Tree

In [49]:
importlib.reload(classifiers)
hypergeometric_simple_clf = classifiers.simple_classifier(hypergeometric, disgenet_labels_df, hypergeometric_test_indices, op_metric='fmeasure')
hypergeometric_simple_clf.to_csv('../../models/apid_huri/disease/disease_hypergeometric_simple.csv', index=False)

  0%|          | 0/223 [00:00<?, ?it/s]

In [50]:
importlib.reload(classifiers)
closeness_simple_clf = classifiers.simple_classifier(closeness, disgenet_labels_df, closeness_test_indices, op_metric='fmeasure')
closeness_simple_clf.to_csv('../../models/apid_huri/disease/disease_closeness_simple.csv', index=False)

  0%|          | 0/223 [00:00<?, ?it/s]

In [51]:
importlib.reload(classifiers)
betweenness_simple_clf = classifiers.simple_classifier(betweenness, disgenet_labels_df, betweenness_test_indices, op_metric='fmeasure')
betweenness_simple_clf.to_csv('../../models/apid_huri/disease/disease_betweenness_simple.csv', index=False)

  0%|          | 0/223 [00:00<?, ?it/s]

In [53]:
importlib.reload(classifiers)
fraction_betweenness_simple_clf = classifiers.simple_classifier(fraction_betweenness, disgenet_labels_df, fraction_betweenness_test_indices, op_metric='fmeasure')
fraction_betweenness_simple_clf.to_csv('../../models/apid_huri/disease/disease_fraction_betweenness_simple.csv', index=False)

  0%|          | 0/223 [00:00<?, ?it/s]

In [54]:
importlib.reload(classifiers)
rwr_simple_clf = classifiers.simple_classifier(rwr, disgenet_labels_df, rwr_test_indices, op_metric='fmeasure')
rwr_simple_clf.to_csv('../../models/apid_huri/disease/disease_rwr_simple.csv', index=False)

  0%|          | 0/223 [00:00<?, ?it/s]

## 1.2 Classification with Conservative Module

In [10]:
importlib.reload(classifiers)
hypergeometric_simple_clf_conservative = classifiers.simple_classifier(hypergeometric_conservative, disgenet_labels_conservative_df, hypergeometric_test_indices_conservative, op_metric='fmeasure')
hypergeometric_simple_clf_conservative.to_csv('../../models/apid_huri/disease/disease_hypergeometric_simple_conservative.csv', index=False)

  0%|          | 0/223 [00:00<?, ?it/s]

In [ ]:
importlib.reload(classifiers)
closeness_simple_clf_conservative = classifiers.simple_classifier(closeness_conservative, disgenet_labels_conservative_df, closeness_test_indices_conservative, op_metric='fmeasure')
closeness_simple_clf_conservative.to_csv('../../models/apid_huri/disease/disease_closeness_simple_conservative.csv', index=False)

In [ ]:
importlib.reload(classifiers)
betweenness_simple_clf_conservative = classifiers.simple_classifier(betweenness_conservative, disgenet_labels_conservative_df, betweenness_test_indices_conservative, op_metric='fmeasure')
betweenness_simple_clf_conservative.to_csv('../../models/apid_huri/disease/disease_betweenness_simple_conservative.csv', index=False)

In [ ]:
importlib.reload(classifiers)
fraction_betweenness_simple_clf_conservative = classifiers.simple_classifier(fraction_betweenness_conservative, disgenet_labels_conservative_df, fraction_betweenness_test_indices_conservative, op_metric='fmeasure')
fraction_betweenness_simple_clf_conservative.to_csv('../../models/apid_huri/disease/disease_fraction_betweenness_simple_conservative.csv', index=False)

In [ ]:
importlib.reload(classifiers)
rwr_simple_clf_conservative = classifiers.simple_classifier(rwr_conservative, disgenet_labels_conservative_df, rwr_test_indices_conservative, op_metric='fmeasure')
rwr_simple_clf_conservative.to_csv('../../models/apid_huri/disease/disease_rwr_simple_conservative.csv', index=False)

# 2. Threshold Classifier - Reduced Networks

## 2.1. PPI 80% Reduction

### 2.1.1. Steinet Tree

In [ ]:
importlib.reload(classifiers)
hyper80_simple_clf = classifiers.reduced_classifiers(2, None, None, ppi80_hyper_list, hypergeometric_80_test_index, None, label_list)
hyper80_simple_clf.to_csv('../../models/apid_huri/disease/disease_hyper80_simple.csv', index=False)

In [ ]:
importlib.reload(classifiers)
rwr80_simple_clf = classifiers.reduced_classifiers(2, None, None, ppi80_rwr_list, rwr_80_test_index, None, label_list)
rwr80_simple_clf.to_csv('../../models/apid_huri/disease/disease_rwr80_simple.csv', index=False)

In [ ]:
importlib.reload(classifiers)
closeness80_simple_clf = classifiers.reduced_classifiers(2, None, None, ppi80_closeness_list, closeness_80_test_index, None, label_list)
closeness80_simple_clf.to_csv('../../models/apid_huri/disease/disease_closeness80_simple.csv', index=False)

In [88]:
importlib.reload(classifiers)
betweenness80_simple_clf = classifiers.reduced_classifiers(2, None, None, ppi80_betweenness_list, betweenness_80_test_index, None, label_list)
betweenness80_simple_clf.to_csv('../../models/apid_huri/disease/disease_betweenness80_simple.csv', index=False)

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/223 [00:00<?, ?it/s]

  0%|          | 0/223 [00:00<?, ?it/s]

  0%|          | 0/223 [00:00<?, ?it/s]

  0%|          | 0/223 [00:00<?, ?it/s]

...

  0%|          | 0/223 [00:00<?, ?it/s]

  0%|          | 0/223 [00:00<?, ?it/s]

  0%|          | 0/223 [00:00<?, ?it/s]

  0%|          | 0/223 [00:00<?, ?it/s]

  0%|          | 0/223 [00:00<?, ?it/s]

In [91]:
importlib.reload(classifiers)
fraction_betweenness80_simple_clf = classifiers.reduced_classifiers(2, None, None, ppi80_fraction_betweenness_list, fraction_betweenness_80_test_index, None, label_list)
fraction_betweenness80_simple_clf.to_csv('../../models/apid_huri/disease/disease_fraction_betweenness80_simple.csv', index=False)

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/223 [00:00<?, ?it/s]

  0%|          | 0/223 [00:00<?, ?it/s]

  0%|          | 0/223 [00:00<?, ?it/s]

  0%|          | 0/223 [00:00<?, ?it/s]

...

  0%|          | 0/223 [00:00<?, ?it/s]

  0%|          | 0/223 [00:00<?, ?it/s]

  0%|          | 0/223 [00:00<?, ?it/s]

  0%|          | 0/223 [00:00<?, ?it/s]

  0%|          | 0/223 [00:00<?, ?it/s]

### 2.1.2. Conservative Module

In [ ]:
importlib.reload(classifiers)
hyper80_simple_clf_conservative = classifiers.reduced_classifiers(2, None, None, ppi80_hyper_list_conservative, hypergeometric_80_test_index_conservative, None, label_list_conservative)
hyper80_simple_clf_conservative.to_csv('../../models/apid_huri/disease/disease_hyper80_simple_conservative.csv', index=False)

In [ ]:
importlib.reload(classifiers)
rwr80_simple_clf_conservative = classifiers.reduced_classifiers(2, None, None, ppi80_rwr_list_conservative, rwr_80_test_index_conservative, None, label_list_conservative)
rwr80_simple_clf_conservative.to_csv('../../models/apid_huri/disease/disease_rwr80_simple_conservative.csv', index=False)

In [ ]:
importlib.reload(classifiers)
closeness80_simple_clf_conservative = classifiers.reduced_classifiers(2, None, None, ppi80_closeness_list_conservative, closeness_80_test_index_conservative, None, label_list_conservative)
closeness80_simple_clf_conservative.to_csv('../../models/apid_huri/disease/disease_closeness80_simple_conservative.csv', index=False)

In [ ]:
importlib.reload(classifiers)
betweenness80_simple_clf_conservative = classifiers.reduced_classifiers(2, None, None, ppi80_betweenness_list_conservative, betweenness_80_test_index_conservative, None, label_list_conservative)
betweenness80_simple_clf_conservative.to_csv('../../models/apid_huri/disease/disease_betweenness80_simple_conservative.csv', index=False)

In [ ]:
importlib.reload(classifiers)
fraction_betweenness80_simple_clf_conservative = classifiers.reduced_classifiers(2, None, None, ppi80_fraction_betweenness_list_conservative, fraction_betweenness_80_test_index_conservative, None, label_list_conservative)
fraction_betweenness80_simple_clf_conservative.to_csv('../../models/apid_huri/disease/disease_fraction_betweenness80_simple_conservative.csv', index=False)

## 2.2. Protein 80% Reduction

### 2.2.1. Steiner Tree

In [94]:
importlib.reload(classifiers)
hyper_protein80_simple_clf = classifiers.reduced_classifiers(2, None, None, protein80_hyper_list, hypergeometric_protein80_test_index, None, label_list_protein80)
hyper_protein80_simple_clf.to_csv('../../models/apid_huri/disease/disease_hyper_protein80_simple.csv', index=False)

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/223 [00:00<?, ?it/s]

  0%|          | 0/223 [00:00<?, ?it/s]

  0%|          | 0/223 [00:00<?, ?it/s]

  0%|          | 0/223 [00:00<?, ?it/s]

...

  0%|          | 0/223 [00:00<?, ?it/s]

  0%|          | 0/223 [00:00<?, ?it/s]

  0%|          | 0/223 [00:00<?, ?it/s]

  0%|          | 0/223 [00:00<?, ?it/s]

  0%|          | 0/223 [00:00<?, ?it/s]

In [95]:
importlib.reload(classifiers)
rwr_protein80_simple_clf = classifiers.reduced_classifiers(2, None, None, protein80_rwr_list, rwr_protein80_test_index, None, label_list_protein80)
rwr_protein80_simple_clf.to_csv('../../models/apid_huri/disease/disease_rwr_protein80_simple.csv', index=False)

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/223 [00:00<?, ?it/s]

  0%|          | 0/223 [00:00<?, ?it/s]

  0%|          | 0/223 [00:00<?, ?it/s]

  0%|          | 0/223 [00:00<?, ?it/s]

...

  0%|          | 0/223 [00:00<?, ?it/s]

  0%|          | 0/223 [00:00<?, ?it/s]

  0%|          | 0/223 [00:00<?, ?it/s]

  0%|          | 0/223 [00:00<?, ?it/s]

  0%|          | 0/223 [00:00<?, ?it/s]

In [96]:
importlib.reload(classifiers)
closeness_protein80_simple_clf = classifiers.reduced_classifiers(2, None, None, protein80_closeness_list, closeness_protein80_test_index, None, label_list_protein80)
closeness_protein80_simple_clf.to_csv('../../models/apid_huri/disease/disease_closeness_protein80_simple.csv', index=False)

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/223 [00:00<?, ?it/s]

  0%|          | 0/223 [00:00<?, ?it/s]

  0%|          | 0/223 [00:00<?, ?it/s]

  0%|          | 0/223 [00:00<?, ?it/s]

...

  0%|          | 0/223 [00:00<?, ?it/s]

  0%|          | 0/223 [00:00<?, ?it/s]

  0%|          | 0/223 [00:00<?, ?it/s]

  0%|          | 0/223 [00:00<?, ?it/s]

  0%|          | 0/223 [00:00<?, ?it/s]

In [97]:
importlib.reload(classifiers)
betweenness_protein80_simple_clf = classifiers.reduced_classifiers(2, None, None, protein80_betweenness_list, betweenness_protein80_test_index, None, label_list_protein80)
betweenness_protein80_simple_clf.to_csv('../../models/apid_huri/disease/disease_betweenness_protein80_simple.csv', index=False)

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/223 [00:00<?, ?it/s]

  0%|          | 0/223 [00:00<?, ?it/s]

  0%|          | 0/223 [00:00<?, ?it/s]

  0%|          | 0/223 [00:00<?, ?it/s]

...

  0%|          | 0/223 [00:00<?, ?it/s]

  0%|          | 0/223 [00:00<?, ?it/s]

  0%|          | 0/223 [00:00<?, ?it/s]

  0%|          | 0/223 [00:00<?, ?it/s]

  0%|          | 0/223 [00:00<?, ?it/s]

In [98]:
importlib.reload(classifiers)
fraction_betweenness_protein80_simple_clf = classifiers.reduced_classifiers(2, None, None, protein80_fraction_betweenness_list, fraction_betweenness_protein80_test_index, None, label_list_protein80)
fraction_betweenness_protein80_simple_clf.to_csv('../../models/apid_huri/disease/disease_fraction_betweenness_protein80_simple.csv', index=False)

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/223 [00:00<?, ?it/s]

  0%|          | 0/223 [00:00<?, ?it/s]

  0%|          | 0/223 [00:00<?, ?it/s]

  0%|          | 0/223 [00:00<?, ?it/s]

...

  0%|          | 0/223 [00:00<?, ?it/s]

  0%|          | 0/223 [00:00<?, ?it/s]

  0%|          | 0/223 [00:00<?, ?it/s]

  0%|          | 0/223 [00:00<?, ?it/s]

  0%|          | 0/223 [00:00<?, ?it/s]

### 2.2.2. Conservative Module

In [99]:
importlib.reload(classifiers)
hyper_protein80_simple_clf_conservative = classifiers.reduced_classifiers(2, None, None, protein80_hyper_list_conservative, hypergeometric_protein80_test_index_conservative, None, label_list_protein80_conservative)
hyper_protein80_simple_clf_conservative.to_csv('../../models/apid_huri/disease/disease_hyper_protein80_simple_conservative.csv', index=False)

NameError: name 'protein80_hyper_list_conservative' is not defined

In [ ]:
importlib.reload(classifiers)
rwr_protein80_simple_clf_conservative = classifiers.reduced_classifiers(2, None, None, protein80_rwr_list_conservative, rwr_protein80_test_index_conservative, None, label_list_protein80_conservative)
rwr_protein80_simple_clf_conservative.to_csv('../../models/apid_huri/disease/disease_rwr_protein80_simple_conservative.csv', index=False)

In [ ]:
importlib.reload(classifiers)
closeness_protein80_simple_clf_conservative = classifiers.reduced_classifiers(2, None, None, protein80_closeness_list_conservative, closeness_protein80_test_index_conservative, None, label_list_protein80_conservative)
closeness_protein80_simple_clf_conservative.to_csv('../../models/apid_huri/disease/disease_closeness_protein80_simple_conservative.csv', index=False)

In [ ]:
importlib.reload(classifiers)
betweenness_protein80_simple_clf_conservative = classifiers.reduced_classifiers(2, None, None, protein80_betweenness_list_conservative, betweenness_protein80_test_index_conservative, None, label_list_protein80_conservative)
betweenness_protein80_simple_clf_conservative.to_csv('../../models/apid_huri/disease/disease_betweenness_protein80_simple_conservative.csv', index=False)

In [ ]:
importlib.reload(classifiers)
fraction_betweenness_protein80_simple_clf_conservative = classifiers.reduced_classifiers(2, None, None, protein80_fraction_betweenness_list_conservative, fraction_betweenness_protein80_test_index_conservative, None, label_list_protein80_conservative)
fraction_betweenness_protein80_simple_clf_conservative.to_csv('../../models/apid_huri/disease/disease_fraction_betweenness_protein80_simple_conservative.csv', index=False)